# Building a LangGraph Agent with Long-Term Memory via Flotorch

This notebook demonstrates how to build an advanced LangGraph agent with both session-based (short-term) and long-term memory capabilities using the Flotorch platform. This allows the agent to recall information from the current conversation as well as from previous, separate interactions, enhanced by custom tools for arithmetic operations.

### Prerequisites
Configure your model, memory provider, and API key in the Flotorch console (https://console.flotorch.cloud/).

### Viewing Logs
Logs can be viewed in the logs tab of the Flotorch console (https://console.flotorch.cloud/).

### Key Concepts:
- **Session-Based Memory (Short-Term)**: Provided by `FlotorchLanggraphSession` for context within the current session.
- **External (Long-Term) Memory**: Provided by `FlotorchLangGraphMemory` for persistent knowledge across all sessions.
- **`create_react_agent`**: LangGraph's function to create a ReAct-style agent with tools and memory integration.
- **Custom Tools**: `multiply` and `addition` tools for performing arithmetic operations.

## 1. Setup and Imports

The following cells install the necessary packages, configure API credentials, and import required components from Flotorch and LangGraph, including those needed for memory and custom tools.

In [ ]:
%pip install flotorch[langgraph]

In [ ]:
FLOTORCH_API_KEY = "<YOUR FLOTORCH_API_KEY>"
FLOTORCH_BASE_URL = "<YOUR FLOTORCH_BASE_URL>"  
FLOTORCH_MODEL_ID = "<YOUR FLOTORCH_MODEL_ID>"
PROVIDER_NAME = "<YOUR PROVIDER_NAME>" #eg : memo-provider
USER_ID = "YOUR USER_ID" #eg : flotorch_user1000
APP_ID = "YOUR APP_ID"   #eg : flotorch_app1000

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from flotorch.langchain.llm import FlotorchLangChainLLM
from flotorch.langgraph.sessions import FlotorchLanggraphSession
from flotorch.langgraph.memory import FlotorchStore

print("✔ Imported necessary libraries successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define a single custom tool using the `@tool` decorator from LangChain. The `multiply` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their product. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.

In [ ]:
@tool
def multiply(numbers: str) -> int:
    """
    Multiply two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The product of the two integers
    """
    print("Multiplying... custom tool")
    a, b = map(int, numbers.split(","))
    return a * b

print("✔ Custom multiplication tool defined successfully.")

## 3. Defining a Custom Tool

To extend the agent's capabilities, we define a custom tool using the `@tool` decorator from LangChain. The `addition` tool takes a comma-separated string of two integers (e.g., 'x,y') and returns their sum. The tool's docstring provides a clear description, enabling the agent to understand its purpose and usage.


In [ ]:
@tool
def addition(numbers: str) -> int:
    """
    Addition of two integers provided as a comma-separated string.

    Args:
        numbers (str): Two integers in the format 'x,y' (e.g., '3,4')

    Returns:
        int: The Addition of the two integers
    """
    print("Adding... custom tool")
    a, b = map(int, numbers.split(","))
    return a + b

tools = [multiply,addition]

print("✔ Custom Addition tool defined successfully.")

## 4. Model Configuration

We initialize the `FlotorchLangChainLLM` from the custom `llm` module, which serves as the reasoning engine for the agent, enabling it to process inputs, invoke tools, and leverage memory.

In [ ]:
model = FlotorchLangChainLLM(
    model_id=FLOTORCH_MODEL_ID,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

print(f"✔ Flotorch LLM model configured")

## 5. Session Storage Setup

We configure the `FlotorchLanggraphSession` as the checkpointer for LangGraph, enabling persistent storage of conversation history within a session. The session is uniquely identified by `app_name`, `user_id`, and `thread_id`.

In [ ]:
checkpointer = FlotorchLanggraphSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    app_name=APP_ID,
    user_id=USER_ID
)

config = {"configurable": {"thread_id": "flotorch-test-thread-1000"}}

print(f"✔ Session storage initialized with thread ID: {config['configurable']['thread_id']}")

## 6. Memory Configuration

- **External (Long-Term) Memory**: Uses `FlotorchLangGraphMemory` to store and retrieve key information across different sessions, acting as the agent's persistent knowledge base.

In [ ]:
memory = FlotorchStore(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    provider_name=PROVIDER_NAME,
    userId=USER_ID,
    appId=APP_ID
)

print("✔ Initialized session-based and long-term memory")

 A custom `search_memories` function enhances the agent's ability to retrieve relevant past information.

In [ ]:
def search_memories(user_message):
    """Search for relevant memories based on user message."""
    try:
        items = memory.search(
            (APP_ID, USER_ID, "memories"),
            query=user_message,
            limit=3
        )
        memories = "\n".join(item.value["text"] for item in items)
        return f"## Relevant Memories\n{memories}" if memories else ""
    except Exception as e:
        print(f"Memory search error: {e}")
        return ""


In [ ]:
def store_interaction_in_memory(user_query, assistant_reply):

    memory.put(
        (APP_ID, USER_ID, "conversations"), "1", 
        {"text": f"User: {user_query}"}
    )

    memory.put(
        (APP_ID, USER_ID, "conversations") ,"2", 
        {"text": f"Assistant: {assistant_reply}"}
    )
    
def enhance_query_with_memory(user_query):

    relevant_memories = search_memories(user_query)
    
    if relevant_memories:
        print(f"Found relevant memories: {relevant_memories}")
        return f"{user_query}\n\n{relevant_memories}"
    else:
        return user_query

print("✔ Improved memory functions defined")


## 7. Agent Configuration

We create a LangGraph agent using `create_react_agent`, integrating the Flotorch LLM, the `add` and `multiply` tools, the `FlotorchLanggraphSession` checkpointer for session-based memory, and the `FlotorchLangGraphMemory` store for long-term memory.

In [ ]:
agent = create_react_agent(
    model=model,
    tools=tools,
    checkpointer=checkpointer,
    store=memory
)

print("✔ LangGraph agent created successfully with session and long-term memory.")

## 8. Interactive Chat

Start an interactive chat session to interact with the agent. Test its ability to perform arithmetic (e.g., 'Multiply 5,6' or 'Addition 3,4') and recall information from both the current session (via `FlotorchLanggraphSession`) and long-term memory (via `FlotorchLangGraphMemory`, e.g., 'What do I love?' or 'What is my favorite programming language?'). Type 'exit' to end.

In [ ]:
while True:
    user_query = input("user: ")
    if user_query.lower().strip() == "exit":
        break
    
    enhanced_message = enhance_query_with_memory(user_query)
    
    response = agent.invoke({"messages": enhanced_message}, config)
    bot_reply = response["messages"][-1].content
    print(f"Assistant: {bot_reply}")

    store_interaction_in_memory(user_query, bot_reply)

print("✔ Interactive session ended.")


## Summary

This notebook detailed the implementation of a sophisticated LangGraph agent equipped with both session-based and long-term memory using Flotorch's infrastructure. The agent recalls information from the current conversation and persists knowledge across sessions, enhanced by custom arithmetic tools.

### Key Achievements

- **Dual-Memory Architecture**  
  Successfully integrated two memory types:  
  - `FlotorchLanggraphSession` for immediate, session-specific context.  
  - `FlotorchLangGraphMemory` for persistent, long-term knowledge.

- **Enhanced Agent Intelligence**  
  The agent demonstrated its ability to perform arithmetic operations (via `multiply` and `addition` tools) and recall facts from a long-term knowledge base (e.g., 'I love pizza'), leading to informed and context-rich responses.
